In [1]:
import numpy as np
import scipy.sparse as sps
import h5py
from termcolor import colored
import os

def decode_str(hdf5_str):
    return hdf5_str.tobytes().decode('utf-16')

# filename = 'data/Head-and-Neck_02.mat'
# filename = 'Prostate_CK/Prostate_CK_03.mat'
filename = 'Liver/Liver_10.mat'
f = h5py.File(filename)
name = decode_str(f['patient']['Identifier'][:])
print(colored('Patient: ' + name, 'blue'))

folder = filename.split('.')[0]
if not os.path.exists(folder):
    os.makedirs(folder)

Patient: Liver 10


In [2]:
class Region:
    def __init__(self, name):
        self.name = name
        self.D = None # Dose deposition matrix (called A in TROTS)

    def compute_dose(self, x):
        self.dose = self.D.dot(x)
        self.min = self.dose.min()
        self.mean = self.dose.mean()
        self.max = self.dose.max()

def load_rois():
    print('Loading dose deposition matrices...')
    
    rois = {}
    for ref in f['patient']['StructureNames'][:]:
        name = decode_str(f[ref[0]][:])
        rois[name] = Region(name)

    # Get the dose matrix for each ROI
    n_mats = f['data']['matrix']['A'].shape[0]
    for i in range(n_mats):
        name = decode_str(f[f['data']['matrix']['Name'][i,0]][:])

        if name in rois:
            roi = rois[name]
            A = f[f['data']['matrix']['A'][i,0]]
            
            if isinstance(A, h5py.Group): # It's a sparse matrix
                data = np.array(A['data']).ravel()
                ir = np.array(A['ir']).ravel()
                jc = np.array(A['jc']).ravel()
                n_voxels = A.attrs.get('MATLAB_sparse')
                n_beamlets = jc.size - 1
                shape = (n_voxels, n_beamlets)
                
                roi.D = sps.csc_matrix((data, ir, jc), shape=shape)
                
                print(colored(f'  {name} -> {shape} (sparse)', 'green'))
            elif isinstance(A, h5py.Dataset): # It's a dense matrix
                roi.D = A[:].T
                shape = roi.D.shape
                print(colored(f'  {name} -> {shape} (dense)', 'green'))
            else:
                print(colored(f'  {name} -> Not processed!', 'magenta'))
        else:
            print(colored(f'  {name} -> Ignored', 'yellow'))

    # Check that all regions have dose matrices
    for name, roi in rois.items():
        if roi.D is None:
            print(colored(f'  {name} -> No dose matrix!', 'red'))
    
    return rois

rois = load_rois()

Loading dose deposition matrices...
  Isocentre -> (1, 1562) (dense)
  PTV -> (5287, 1562) (dense)
  Stomach -> (5168, 1562) (sparse)
  Spinal Cord -> (5155, 1562) (sparse)
  Heart -> (5152, 1562) (sparse)
  Oesophagus -> (3511, 1562) (sparse)
  Duodenum -> (5135, 1562) (sparse)
  Bowels -> (5127, 1562) (sparse)
  External Ring -> (4851, 1562) (sparse)
  Liver minus CTV -> (5121, 1562) (sparse)
  Kidney (R) -> (5267, 1562) (sparse)
  Kidney (L) -> (5253, 1562) (sparse)
  Isocentre -> (1, 1562) (dense)
  Liver minus CTV (mean) -> Ignored
  Stomach (mean) -> Ignored
  Duodenum (mean) -> Ignored
  Oesophagus (mean) -> Ignored
  Heart (mean) -> Ignored
  Spinal Cord (mean) -> Ignored
  Bowels (mean) -> Ignored
  Kidney (R) (mean) -> Ignored
  Kidney (L) (mean) -> Ignored
  Patient (mean) -> Ignored
  Patient -> (29549, 1562) (sparse)
  Smoothing Linear -> Ignored
  Smoothing Quadratic -> Ignored
  Sm2 beam 1 -> Ignored
  Sm2 beam 2 -> Ignored
  Sm2 beam 3 -> Ignored
  Sm2 beam 4 -> Ignored

In [3]:
import os

def n_voxels(rois):
    n_voxels = 0
    for name, roi in rois.items():
        n_voxels += roi.D.shape[0]
    return n_voxels

def gen_m(folder):
    name = decode_str(f['patient']['Identifier'][:]).replace(' ', '_')
    with open(os.path.join(folder, 'm_' + name + '.txt'), 'w') as file:
        file.write(name + '\n')
        # We don't care about beam geometry for now.
        file.write('1 // Number of beams \n')
        file.write('1 {} \n'.format(int(f['data']['misc']['size'][0][0])))
        file.write('{} // Number of voxels \n'.format(n_voxels(rois)))
        # d_ files will hold float values without need of scaling.
        file.write('1.0 // DoseGridScaling \n'.format(n_voxels(rois)))
        file.write('{} // Number of ROIs \n'.format(len(rois)))
        for i, (name, roi) in enumerate(rois.items()):
            file.write('{} {}\n'.format(2**i, name))
            

def gen_v(folder):
    name = decode_str(f['patient']['Identifier'][:]).replace(' ', '_')
    with open(os.path.join(folder, 'v_' + name + '.txt'), 'w') as file:
        for i, (name, roi) in enumerate(rois.items()):
            print(name)
            voxel_region = 2**i
            for j in range(roi.D.shape[0]):
                file.write('{}\n'.format(voxel_region))
                
gen_m(folder)
gen_v(folder)

PTV
Stomach
Duodenum
Oesophagus
Heart
Spinal Cord
Kidney (R)
Kidney (L)
Bowels
Liver minus CTV
Patient
Isocentre
External Ring
PTV40%
PTV80%
Ring 1


In [4]:
def gen_d(folder):
    name = decode_str(f['patient']['Identifier'][:]).replace(' ', '_')
    
    total_nnz = 0
    for n, roi in rois.items():
        coo = sps.coo_matrix(roi.D)
        total_nnz += coo.nnz
    
    with open(os.path.join(folder, 'd_' + name + '.txt'), 'w') as file:
        file.write('{} \n'.format(total_nnz))
        row_offset = 0
        for i, (n, roi) in enumerate(rois.items()):
            coo = sps.coo_matrix(roi.D)
            print(f'[{i+1:02d}/{len(rois):02d}] {n}: {coo.nnz} nonzeros...')
            for j in range(coo.nnz):
                file.write('{} {} {:.8f}\n'.format(coo.row[j] + row_offset, coo.col[j], coo.data[j]))
            row_offset += coo.shape[0]

gen_d(folder)

[01/16] PTV: 7360506 nonzeros...
[02/16] Stomach: 521374 nonzeros...
[03/16] Duodenum: 682591 nonzeros...
[04/16] Oesophagus: 966459 nonzeros...
[05/16] Heart: 350197 nonzeros...
[06/16] Spinal Cord: 527781 nonzeros...
[07/16] Kidney (R): 1247675 nonzeros...
[08/16] Kidney (L): 496681 nonzeros...
[09/16] Bowels: 328933 nonzeros...
[10/16] Liver minus CTV: 1757033 nonzeros...
[11/16] Patient: 1309048 nonzeros...
[12/16] Isocentre: 1271 nonzeros...
[13/16] External Ring: 143475 nonzeros...
[14/16] PTV40%: 7399841 nonzeros...
[15/16] PTV80%: 7366883 nonzeros...
[16/16] Ring 1: 6935212 nonzeros...


In [5]:
def gen_x():
    name = decode_str(f['patient']['Identifier'][:]).replace(' ', '_')
    fluence = f['solutionX'][:].ravel()
    
    with open(os.path.join(folder, 'x_' + name + '.txt'), 'w') as file:
        for x in fluence:
            file.write('{}\n'.format(x))

gen_x()
